<a href="https://colab.research.google.com/github/JEETMANISHBOBDE/Image-Restoration/blob/main/Image_restoration.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import files
import zipfile
import io
import os

# Upload file(s)
uploaded = files.upload()

# Create a directory to store the data
data_dir = 'data'
if not os.path.exists(data_dir):
    os.makedirs(data_dir)


